In [1]:
  # import libraries
try:
    # %tensorflow_version only exists in Colab.
    !pip install tf-nightly
except Exception:
    pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.6/556.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.4/439.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.15.0
    Uninstalling absl-py-0.15.0:
      Successfully uninstalled absl-py-0.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

2022-12-31 13:02:30.316430: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-31 13:02:30.590842: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-12-31 13:02:30.590907: I tensorflow/tsl/cuda/cudart_stub.cc:28] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-31 13:02:30.636737: E tensorflow/tsl/lib/monitoring/collection_registry.cc:81] Cannot register 2 metrics with the same name: /tensorflow/core/bfc_a

2.12.0-dev20221213


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-12-31 13:02:48--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2022-12-31 13:02:48 (7.64 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-12-31 13:02:49--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
traindata = pd.read_table(train_file_path)
vailddata = pd.read_table(test_file_path)

In [4]:
vailddata.head()

,ham,i am in hospital da. . i will return home in evening
0,ham,"not much, just some textin'. how bout you?"
1,ham,i probably won't eat at all today. i think i'm...
2,ham,don‘t give a flying monkeys wot they think and...
3,ham,who are you seeing?
4,ham,your opinion about me? 1. over 2. jada 3. kusr...


In [5]:
#i think by doing columns this way i lose the first entry but well its not a big deal i think
traindata.columns = ['Classification', 'message']
vailddata.columns = ['Classification', 'message']

traindata.head()

,Classification,message
0,ham,you can never do nothing
1,ham,"now u sound like manky scouse boy steve,like! ..."
2,ham,mum say we wan to go then go... then she can s...
3,ham,never y lei... i v lazy... got wat? dat day ü ...
4,ham,in xam hall boy asked girl tell me the startin...


In [6]:
fulldata = traindata.message.append(vailddata.message, ignore_index=True) #i combine both datasets because i want to fill tokenizer (which i explain later) on entire data


In [7]:
a = fulldata.str.split().apply(len) #Later i will need to know how many words emailes have, because i need them to have the same number of words i think good idea is to take mean number of words and trim excess words

In [8]:
print(max(a))
print(a.quantile(0.75)) #actually i decided to go with 3th quantile, where 75% of data is below this number and only 25% is excess

length_of_emails = int(a.quantile(0.75))

171
23.0


In [9]:
print(len(fulldata)) #not important but we have 5569 messages in total 

5569


In [10]:
from keras.preprocessing.text import Tokenizer

# Initialize tokenizer, what this does is essentialy it maps each word to some number, so we get a full vocabulary of numbers where each is some word. its important cuz you can,t do anything with text strings
tokenizer = Tokenizer()

# Fit the Tokenizer on the entire data so it has all the words in vocab
tokenizer.fit_on_texts(fulldata)

# Convert the training data to sequences of tokens
train_X = tokenizer.texts_to_sequences(traindata.message)

# Convert the validation data to sequences of tokens
val_X = tokenizer.texts_to_sequences(vailddata.message)

In [11]:
print(val_X[0]) #it looks like this
print(vailddata.message.loc[0])

[25, 122, 37, 116, 7819, 50, 742, 3]
not much, just some textin'. how bout you?


In [12]:
#now i use this method to trim any email that has more than a mean amount of words in data (16) , padding fills this array of emails as numbers with 0, where 'post' means fill at the end if its less than 16. 
#truncating trims emails with more than 16 words, again post meaning trim at the end.  'pre' would trim at the beginning.
train_X = keras.preprocessing.sequence.pad_sequences(train_X, maxlen=length_of_emails, padding='post', truncating='post') 
val_X =  keras.preprocessing.sequence.pad_sequences(val_X, maxlen=length_of_emails, padding='post', truncating='post')

In [13]:
print(val_X) #weird but ok

[[  25  122   37 ...    0    0    0]
 [   1  388  569 ...    0    0    0]
 [1683  138    4 ...    0    0    0]
 ...
 [ 131   22    3 ...    4 8993   20]
 [  33  158  856 ... 1686 8994 1020]
 [  25  958   47 ... 1083 4419   57]]


In [14]:
vocab_size = len(tokenizer.word_index) + 1 #soo here is how to get vocabulary size out of this tokenizer. as you see it has more entries than fulldata words but this is because it has all the unique words, earlier we counted
#just a total number of words
print(vocab_size)

8995


In [15]:

train_Y = traindata.Classification #remove labels from training and validation data
val_Y = vailddata.Classification

In [16]:
val_Y.replace(to_replace = ['ham','spam'], value = [0,1],inplace = True) #now labels are string, again they need to be numbers so replace ham with 0 and spam with 1
train_Y.replace(to_replace = ['ham','spam'], value = [0,1], inplace = True)
train_Y.head()

0    0
1    0
2    0
3    0
4    0
Name: Classification, dtype: int64

In [17]:
from sklearn.model_selection import train_test_split #its good to have a test data so i take a small fraction of train data to get test data by using this method, pretty self explainatory.
train_X, test_X, train_Y, test_Y = train_test_split(train_X, train_Y, random_state=2, test_size = 0.1) 

In [18]:
print(len(train_X))
print(len(test_X))
print(len(val_X))

3760
418
1391


In [19]:
#okay the model, 
model = keras.Sequential([
    
  keras.layers.Embedding(input_dim=vocab_size, output_dim=75, input_length=length_of_emails), #generally it preprocess the data so the integers are mapped into vectors , output dim lets you choose how many numbers this vector has
  keras.layers.LSTM(units=256, dropout=0.3, recurrent_dropout=0.3), #
  keras.layers.Dense(units=1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.001),metrics=[ 'acc'])
model.summary()

2022-12-31 13:02:50.837729: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-12-31 13:02:50.837782: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-31 13:02:50.837808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (6e6a0bc69f35): /proc/driver/nvidia/version does not exist
2022-12-31 13:02:50.838263: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild Tenso

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 75)            674625    
                                                                 
 lstm (LSTM)                 (None, 256)               339968    
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 1,014,850
Trainable params: 1,014,850
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(train_X, train_Y, epochs=10, batch_size = 32,validation_data=(val_X, val_Y)) #train the data

Epoch 1/10
118/118 [==============================] - 18s 127ms/step - loss: 0.1566 - acc: 0.9463 - val_loss: 0.0724 - val_acc: 0.9784
Epoch 2/10
118/118 [==============================] - 20s 170ms/step - loss: 0.0393 - acc: 0.9894 - val_loss: 0.0773 - val_acc: 0.9777
Epoch 3/10
118/118 [==============================] - 16s 137ms/step - loss: 0.0171 - acc: 0.9960 - val_loss: 0.0569 - val_acc: 0.9871
Epoch 4/10
118/118 [==============================] - 15s 131ms/step - loss: 0.0086 - acc: 0.9979 - val_loss: 0.0547 - val_acc: 0.9835
Epoch 5/10
118/118 [==============================] - 14s 117ms/step - loss: 0.0126 - acc: 0.9976 - val_loss: 0.0586 - val_acc: 0.9842
Epoch 6/10
118/118 [==============================] - 13s 112ms/step - loss: 0.0028 - acc: 0.9995 - val_loss: 0.0499 - val_acc: 0.9863
Epoch 7/10
118/118 [==============================] - 14s 119ms/step - loss: 4.3608e-04 - acc: 1.0000 - val_loss: 0.1019 - val_acc: 0.9849
Epoch 8/10
118/118 [==============================]

In [21]:
y_pred = model.predict(test_X) #predictions on test data to see how good model is later
y_prediction = []
for i in y_pred:
  if i > 0.3:
    y_prediction.append(1)
  else:
    y_prediction.append(0)


14/14 [==============================] - 1s 19ms/step


In [22]:
print(y_prediction)
print(test_Y)

[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [23]:
from sklearn.metrics import precision_recall_fscore_support

# so this function allows me to see precision and recall
#precision is basically number of  true positive (predictions belonging to class closer to 1 in binary classification , which are correct)/all the positives model made eg. 80/100 means 80% precision and that 80 of 100 predictions for eg of spam were correct.
#recall is  number of true positves but this time divided by an actual number of positives so lets say we have 100 spam emails. if model predicts 70 of them then recall is 70/100 = 70%
#f1 score is just some math including previous metrics considered as good indicator 
precision, recall, f1_score, support = precision_recall_fscore_support(test_Y, y_prediction, average='binary')

print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')

F1 = 2 * (precision * recall) / (precision + recall)
print(f'F1 score the (higher the better between 0 and 1) is:  {f1_score:.3f}')

#seems good

#IMPORTANT, due to this task making me predict specific emails, i see that when i set the threshold of whether an email is a spam or not to 0.3 i cannot complete the task but it typically has a better F1 score.
#when i set it to 0.0001 the task is completed but the F1 score drops

Precision: 0.887
Recall: 0.948
F1 score the (higher the better between 0 and 1) is:  0.917


In [24]:

pred_text = ["how are you doing today?"] #it needs to be in a list or tokenizer encodes each letter
pred_text  = tokenizer.texts_to_sequences(pred_text)
#pred_text = [[elem for sublist in pred_text for elem in sublist]] it flattened some version of a list no longer neccasary
print(pred_text)
pred_text = keras.preprocessing.sequence.pad_sequences(pred_text, maxlen=16, padding='post', truncating='post') #each text to predict has to be trimmed aswell

print(pred_text)

#just testing how to do function they want same code below 

[[50, 22, 3, 159, 89]]
[[ 50  22   3 159  89   0   0   0   0   0   0   0   0   0   0   0]]


In [25]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text  = tokenizer.texts_to_sequences([pred_text])

  #pred_text = [[elem for sublist in pred_text for elem in sublist]]
  print(pred_text)
  pred_text = keras.preprocessing.sequence.pad_sequences(pred_text, maxlen=16, padding='post', truncating='post')

  #print(pred_text)

  prediction = model.predict(pred_text)
  #print(prediction) its in a list in a list (2d list) thats why double index to take the value below
  if prediction[0][0] > 0.3: #this threshold seems to work the best (0.3)
    label = 'spam'
  else:
    label = 'ham'
  print(prediction[0][0], label) 
  return (prediction[0][0], label)

pred_text = "hello your computer has virus pay 10 dollars to get this for free" 
prediction = predict_message(pred_text)
print(prediction)

[[258, 13, 932, 118, 394, 431, 1692, 2, 31, 40, 12, 48]]
1/1 [==============================] - 0s 259ms/step
0.661164 spam
(0.661164, 'spam')


In [26]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


[[50, 22, 3, 159, 89]]
1/1 [==============================] - 0s 30ms/step
0.000698405 ham
[[1248, 89, 2, 84, 469, 16]]
1/1 [==============================] - 0s 32ms/step
0.9995712 spam
[[1, 93, 70, 2, 49, 28, 39, 261, 14, 4, 1068, 64, 663, 408]]
1/1 [==============================] - 0s 27ms/step
0.006807847 ham
[[92, 102, 97, 375, 230, 9, 287, 37, 8433, 18, 13, 111, 2, 345, 379]]
1/1 [==============================] - 0s 28ms/step
0.9999323 spam
[[3, 17, 185, 370, 158, 16, 2, 128, 13, 152]]
1/1 [==============================] - 0s 33ms/step
0.9999347 spam
[[79, 498, 14, 151, 98, 665, 5, 2287]]
1/1 [==============================] - 0s 28ms/step
0.0009531549 ham
[[926, 9, 13, 4283, 530, 20, 728, 2, 10, 74, 63, 134]]
1/1 [==============================] - 0s 30ms/step
0.00036374907 ham
You passed the challenge. Great job!
